In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import MeanShift, estimate_bandwidth

# Convert dat to cvs

In [2]:
# Функция возвращает является ли значение числом (float)
def isfloat(value):
  try:
    float(value)
    return True
  except:
    return False
    
    
# открываем исходный файл, создаем новый csv
file_dat=open('checkins.dat')
file_csv=open('checkins.csv',mode='w')
# Добавляем заголовок 
file_csv.write("id,user_id,venue_id,latitude,longitude,created_at\n")
# Читаем исходный файл, проверяем есть ли в строке координаты, если да, то записываем в csv
for line in file_dat:
    line = line.split('|')
    # Проверяем что есть значения
    if len(line)!=6:
        continue    
    # Проверяем что заданы координаты
    if not(isfloat(line[3]) & isfloat(line[4])):
        continue
    line = [x.strip() for x in line]
    line = ",".join(line)
    file_csv.write(line+'\n')
file_dat.close()
file_csv.close()

FileNotFoundError: [Errno 2] No such file or directory: 'checkins.dat'

In [ ]:
file_dat.close()
file_csv.close()

# Загружаем даные из csv файла

In [3]:
df=pd.read_csv('checkins.csv')
df['created_at']=pd.to_datetime(df['created_at'])
df.head()

,id,user_id,venue_id,latitude,longitude,created_at
0,984222,15824,5222,38.895112,-77.036366,2012-04-21 17:43:47
1,984234,44652,5222,33.800745,-84.410520,2012-04-21 17:43:43
2,984291,105054,5222,45.523452,-122.676207,2012-04-21 17:39:22
3,984318,2146539,5222,40.764462,-111.904565,2012-04-21 17:35:46
4,984232,93870,380645,33.448377,-112.074037,2012-04-21 17:38:18


# Кластеризация

In [4]:
# Оставляем 100 000 точек
X=df[['latitude','longitude']]
X=X[0:100000]

In [ ]:
# Формируем кластеры
ms = MeanShift(bandwidth=0.1)
ms.fit(X)
labels = ms.labels_
cluster_centers = ms.cluster_centers_

labels_unique = np.unique(labels)
n_clusters_ = len(labels_unique)

print("number of estimated clusters : %d" % n_clusters_)

## Оставляем только значимые кластеры

In [ ]:
ul,uc=np.unique(labels,return_counts=True)
import_clusers=cluster_centers[uc>15]
# Записываем их в файл
np.savetxt('task1_import_clusers.csv', import_clusers, delimiter=",",fmt='%1.7f')
import_clusters=pd.DataFrame(import_clusers,columns=['latitude','longitude'])

In [348]:
# Офисы компании
offices=np.array([[33.751277, -118.188740],
                [25.867736, -80.324116],
                [51.503016, -0.075479],
                [52.378894, 4.885084],
                [39.366487, 117.036146],
                [-33.868457, 151.205134]])

In [349]:
def get_dist(x,y):
    min_dist=1000000
    for office in offices:
        dist=((office[0]-x)**2+(office[1]-y)**2)**0.5
        if dist<min_dist:
            min_dist=dist
    return min_dist
import_clusters['MinDist']=import_clusters.apply(lambda row: get_dist(row['latitude'], row['longitude']), axis=1)
near_clusters=import_clusters.sort_values(by=['MinDist'])[0:20]
near_clusters.to_csv('task1_near_clusters.csv',header=False,index=False,columns=['latitude','longitude'])

# Ответ на задание

In [351]:
nearest=near_clusters[0:1].loc[:,['latitude', 'longitude']].values
np.savetxt('task1_result.csv',nearest,delimiter=' ',fmt='%1.7f')

In [ ]:
-33.8606304 151.2047759